In [6]:
import numpy as np
import utilities as uti
from plyfile import PlyData, PlyElement

import sys
sys.path.insert(1, './Colors')
import realman_gaussian_color as rgc
import gradient_color as gdc
import gaussian_color as gc

curvarute_type = 'gauss'
cam = np.array([0,0,0])
height = 400
width = int(height*0.5)
poses = 24

In [8]:
from PIL import Image
def projectImage(data,direction=False):
    frame = np.zeros((height, width, 3), dtype=np.uint8)
    frame[:,:] = np.array([0,0,255])
    for i in range(len(data)):
        for j in range(len(data[0])):
            if not np.isnan(data[i][j]):
                frame[i][j] = np.array([0,0,0.5])*255
    img = Image.fromarray(frame, 'HSV')
    if direction:
        img.save(direction)
    img.show()
    return frame

In [9]:
def interpolate(raw,radius=1):
    data = raw.copy()
    h = len(data)
    w = len(data[0])
    for i in range(h):
        for j in range(w):
            if getBorder(i,j,data) and np.isnan(data[i][j]):
                subarray = data[i-radius:i+radius+1,j-radius:j+radius+1]
                if np.count_nonzero(~np.isnan(subarray))<1:
                    print("point :"+str(i)+","+str(j)+" is isolated!")
                else:
                    data[i][j] = np.nansum(subarray)/np.count_nonzero(~np.isnan(subarray))
    return data

def getBorder(x,y,data):
    row = data[x]
    col = data[:,y]
    top = [np.NaN,np.NaN]
    bot = [np.NaN,np.NaN]
    right = [np.NaN,np.NaN]
    left = [np.NaN,np.NaN]
    h = len(data)
    w = len(data[0])
    for i in range(h):
        if np.isnan(top[1]):
            top[1] = col[i]
            top[0] = i
        if np.isnan(bot[1]):
            bot[1] = col[h - 1 - i]
            bot[0] = h -1 - i
    for i in range(w):
        if np.isnan(left[1]):
            left[1] = row[i]
            left[0] = i
        if np.isnan(right[1]):
            right[1] = row[w - 1 - i]
            right[0] = w - 1 - i
    return (left[0]<y<right[0]) and (top[0]<x<bot[0])

## Init for real data
    def init(index):
        plydata = PlyData.read('./Mesh/results/jean normal/nonback_'+curvarute_type+'_pose_'+str(index)+'.ply')
        points = np.zeros((plydata['vertex'].count,2))
        values = plydata['vertex']['quality']
        points[:,0] = plydata['vertex']['x']
        points[:,1] = plydata['vertex']['y']
        center = [np.average(points[:,0]),np.average(points[:,1])]
        points[:,0] = np.around((points[:,0] - center[0])*100 + width/2)
        points[:,1] = np.around((points[:,1] - center[1])*100 + height/2)
        return points,values

In [10]:
## Init for simulation
    def init(index):
        dic = './Mesh/results/24 frames/fg'+str(index)+'.ply'
        points,values = uti.filterBack(dic)
        center = [np.average(points[:,0]),np.average(points[:,1])]
        points[:,0] = np.around((points[:,0] - center[0])*10*2 + width/2)       # axis x
        points[:,1] = np.around((points[:,1] - center[1])*10*2 + height/2 + 40)    # axix y and 20 is the space at bottom
        return points,values

In [11]:
# Main
frame = np.empty((height, width))
frame[:,:] = np.NaN
for i in range(poses):
    points,values = init(i+1)
    for i in range(len(values)):
        if np.isnan(frame[height-int(points[i][1])][int(points[i][0])]):
            frame[height-int(points[i][1])][int(points[i][0])] = values[i]
        else:
            frame[height-int(points[i][1])][int(points[i][0])] += values[i]
    # # interpolate before calc average
    # frame = interpolate(frame,3)

    import pandas as pd
    df = pd.DataFrame(data=frame)
    alter = df.interpolate(method='nearest').to_numpy()

In [12]:
result = projectImage(frame)

In [ ]:
> The overriding design goal for Markdown's
> formatting syntax is to make it as readable
> as possible. The idea is that a
> Markdown-formatted document should be
> publishable as-is, as plain text, without
> looking like it's been marked up with tags
> or formatting instructions.

    #alter = interpolate(raw,2)
    alter = np.nan_to_num(raw)
    frame2 = projectImage(alter,'./Images/gauss_sim_fixed_24frames.png')